### Parsl Workflow with Ingmar's lake Change Sample Data 

- gpkg files
- new TMS to work with GeoPandas

In [1]:
# file paths
import os
from pathlib import Path

# visualization
import matplotlib.pyplot as plt
import geopandas as gpd
from shapely.geometry import box

# PDG packages
import pdgstaging
import pdgraster
#import py3dtiles
#import viz_3dtiles
#from viz_3dtiles import TreeGenerator, BoundingVolumeRegion
#import pdgpy3dtiles
#from StagedTo3DConverter import StagedTo3DConverter

# logging and configuration
from datetime import datetime
import logging
import logging.config
import argparse
import json

# Parsl
import parsl
from parsl import python_app
from parsl.config import Config
from parsl.channels import LocalChannel
from parsl.executors import HighThroughputExecutor
from parsl.providers import LocalProvider


### Configuration

Note: Robyn figured out that using TMS WGS1984Quad instead of WorldCRS84Quad fixes the geopandas issue. This "new" TMS is specified in the config.\
Config sourced in for this workflow: `ingmar_config.json`

```
{
    "simplify_tolerance": 0.0001,
    "tms_id": "WGS1984Quad",
    "z_range": [0, 11],
    "statistics": [
        {
            "name": "polygon_count",
            "weight_by": "count",
            "property": "centroids_per_pixel",
            "aggregation_method": "sum",
            "resampling_method": "sum",
            "val_range": [0, null],
            "nodata_val": 0,
            "nodata_color": "#ffffff00",
            "palette": ["#d9c43f", "#d93fce"]
        },
        {
            "name": "coverage",
            "weight_by": "area",
            "property": "area_per_pixel_area",
            "aggregation_method": "sum",
            "resampling_method": "average",
            "val_range": [0, 1],
            "nodata_val": 0,
            "nodata_color": "#ffffff00",
            "palette": ["#d9c43f", "#d93fce"]
        }
    ],
    "deduplicate_at": ["staging"],
    "deduplicate_method": "neighbor",
    "deduplicate_keep_rules": [["staging_filename", "larger"]],
    "deduplicate_overlap_tolerance": 0.1,
    "deduplicate_overlap_both": false,
    "deduplicate_centroid_tolerance": null
  }
  ```

### Python Environment

Using jcohen's virtual env `pdgWorkflow` with `parsl` and updated `pdgstaging` & `pdgraster` as of 12/5/22.

Packages & versions used for this workflow:

```
Package            Version
------------------ ----------
affine             2.3.1
asttokens          2.0.5
attrs              22.1.0
backcall           0.2.0
bcrypt             4.0.1
certifi            2022.9.24
cffi               1.15.1
charset-normalizer 2.1.1
click              8.1.3
click-plugins      1.1.1
cligj              0.7.2
coloraide          0.18.1
colormaps          0.3
contourpy          1.0.6
cryptography       38.0.4
cycler             0.11.0
debugpy            1.5.1
decorator          5.1.1
dill               0.3.6
entrypoints        0.4
executing          0.8.3
filelock           3.8.2
Fiona              1.8.22
fonttools          4.38.0
geopandas          0.12.1
globus-sdk         3.15.0
idna               3.4
ipykernel          6.15.2
ipython            8.6.0
jedi               0.18.1
jupyter_client     7.4.7
jupyter_core       4.11.2
kiwisolver         1.4.4
matplotlib         3.6.2
matplotlib-inline  0.1.6
morecantile        3.2.2
munch              2.5.0
nest-asyncio       1.5.5
numpy              1.23.5
packaging          21.3
pandas             1.5.2
paramiko           2.12.0
parsl              2022.11.28
parso              0.8.3
pdgraster          0.1.0
pdgstaging         0.1.0
pexpect            4.8.0
pickleshare        0.7.5
Pillow             9.3.0
pip                22.2.2
prompt-toolkit     3.0.20
psutil             5.9.4
ptyprocess         0.7.0
pure-eval          0.2.2
pycparser          2.21
pydantic           1.10.2
Pygments           2.11.2
PyJWT              2.6.0
PyNaCl             1.5.0
pyparsing          3.0.9
pyproj             3.4.0
python-dateutil    2.8.2
pytz               2022.6
pyzmq              24.0.1
rasterio           1.3.4
requests           2.28.1
Rtree              0.9.7
setproctitle       1.3.2
setuptools         65.5.0
shapely            2.0rc2
six                1.16.0
snuggs             1.4.7
stack-data         0.2.0
tblib              1.7.0
tornado            6.2
traitlets          5.1.1
typeguard          2.13.3
typing_extensions  4.4.0
urllib3            1.26.13
wcwidth            0.2.5
wheel              0.37.1
```

For problem file exploration during rasterization, uninstalled `pdgraster` and installed package with local edits.

### Lake change sample files

In [2]:
base_dir = Path('/home/pdg/data/nitze_lake_change/data_sample_2022-09-09')
#subdirs = ['32607', '32608', '32609']
filename = 'lake_change.gpkg'
#to define each .gpkg file within each UTM subdir as a string representation with forward slashes, use as_posix() for each iteration
#of base_dir + filename. The ** represents that any subdir string can be present between the base_dir and the filename
data_paths = [p.as_posix() for p in base_dir.glob('**/' + filename)]
data_paths

# smallest data sample for troubleshooting, 2 gpkg files with spatial overlap:
# file1 = '/home/thiessenbock/PDG-test/minimal-example/input/file1.gpkg'
# file2 = '/home/thiessenbock/PDG-test/minimal-example/input/file2.gpkg'
# data_paths = [file1, file2]
# data_paths

['/home/pdg/data/nitze_lake_change/data_sample_2022-09-09/32609/05_Lake_Dataset_Raster_02_final/lake_change.gpkg',
 '/home/pdg/data/nitze_lake_change/data_sample_2022-09-09/32608/05_Lake_Dataset_Raster_02_final/lake_change.gpkg',
 '/home/pdg/data/nitze_lake_change/data_sample_2022-09-09/32607/05_Lake_Dataset_Raster_02_final/lake_change.gpkg']

In [3]:
workflow_config = '/home/jcohen/viz-workflow/workflow_troubleshooting/ingmar_config.json'

# logging setup
logging_config = '/home/jcohen/viz-workflow/workflow_troubleshooting/logging.json'

def setup_logging(log_json_file):
    """
    Setup logging configuration
    """
    with open(log_json_file, 'r') as f:
        logging_dict = json.load(f)
    logging.config.dictConfig(logging_dict)
    return logging_dict

logging_dict = setup_logging(logging_config)

logger = logging.getLogger(__name__)

start_time = datetime.now()

In [4]:
class StagedTo3DConverter():
    """
        Processes staged vector data into Cesium 3D tiles according to the
        settings in a config file or dict. This class acts as the orchestrator
        of the other viz-3dtiles classes, and coordinates the sending and
        receiving of information between them.
    """

    def __init__(
        self,
        config
    ):
        """
            Automatically initialize the StagedTo3DConverter class by appying the configuration when an object of that class is created.

            Parameters
            ----------
            self : need to explicitly state this parameter to pass any newly created object of class StagedTo3DConverter to the other paraneter (config)
                this is a python syntax requirement in order for the object to persist of this class

            config : dict or str
                A dictionary of configuration settings or a path to a config
                JSON file. (See help(pdgstaging.ConfigManager))

            Notes
            ----------
            - this function does not do the staging or tiling steps
        """

        self.config = pdgstaging.ConfigManager(config)
        self.tiles = pdgstaging.TilePathManager(
            **self.config.get_path_manager_config())

    def all_staged_to_3dtiles(
        self
    ):
        """
            Process all staged vector tiles into 3D tiles. This is simply a loop that iterates the function staged_to_rdtile() over all files in the staged directory.
        """

        # Get the list of staged vector tiles
        paths = self.tiles.get_filenames_from_dir('staged')
        # Process each tile
        for path in paths:
            self.staged_to_3dtile(path)

    def staged_to_3dtile(self, path):
        """
            Convert a staged vector tile into a B3DM tile file and a matching
            JSON tileset file.
            - the B3DM tile is applied to the PDG portal for visualization purposes
            - the JSON serves as the metadata for that tile

            Parameters
            ----------
            path : str
                The path to the staged vector tile.

            Returns
            -------
            tile, tileset : Cesium3DTile, Tileset
                The Cesium3DTiles and Cesium3DTileset objects
        """

        try:
            
            # Get information about the tile from the path
            tile = self.tiles.tile_from_path(path)
            out_path = self.tiles.path_from_tile(tile, '3dtiles')

            tile_bv = self.bounding_region_for_tile(tile) # bv = bounding volumne

            # Get the filename of the tile WITHOUT the extension
            tile_filename = os.path.splitext(os.path.basename(out_path))[0]
            # Get the base of the path, without the filename
            tile_dir = os.path.dirname(out_path) + os.path.sep

            # Log the event
            logger.info(
                f'Creating 3dtile from {path} for tile {tile} to {out_path}.')

            # Read in the staged vector tile
            gdf = gpd.read_file(path)

            # Summary of following steps:
            # Now that we have the path to the staged vector tile esptablished and logged, 
            # the following checks are executed on each staged vector tile:
            # 1. check if the tile has any data to start with
            # 2. check if the centroid of the polygons within the tile are within the tile boundaries, remove if not
            # 3. check if polygons within the tile overlap, deduplicate them if they do
            # 4. check if the tile has any data left if deduplication was executed
            # 5. if there were errors in the above steps, log that for debugging

            
            # Check if the gdf is empty
            if len(gdf) == 0:
                logger.warning(
                    f'Vector tile {path} is empty. 3D tile will not be'
                    ' created.')
                return

            # Remove polygons with centroids that are outside the tile boundary
            prop_cent_in_tile = self.config.polygon_prop(
                'centroid_within_tile')
            gdf = gdf[gdf[prop_cent_in_tile]]

            # Check if deduplication should be performed
            dedup_here = self.config.deduplicate_at('3dtiles')
            dedup_method = self.config.get_deduplication_method()

            # Deduplicate if required
            if dedup_here and (dedup_method is not None):
                dedup_config = self.config.get_deduplication_config(gdf)
                dedup = dedup_method(gdf, **dedup_config)
                gdf = dedup['keep']

                # The tile could theoretically be empty after deduplication
                if len(gdf) == 0:
                    logger.warning(
                        f'Vector tile {path} is empty after deduplication.'
                        ' 3D Tile will not be created.')
                    return

            # Create & save the b3dm file
            ces_tile, ces_tileset = TreeGenerator.leaf_tile_from_gdf(
                gdf,
                dir=tile_dir,
                filename=tile_filename,
                z=self.config.get('z_coord'),
                geometricError=self.config.get('geometricError'),
                tilesetVersion=self.config.get('version'),
                boundingVolume=tile_bv
            )

            return ces_tile, ces_tileset

        except Exception as e:
            logger.error(f'Error creating 3D Tile from {path}.')
            logger.error(e)

    def parent_3dtiles_from_children(self, tiles, bv_limit=None):
        """
            Create parent Cesium 3D Tileset json files that point to
            of child JSON files in the tile tree hierarchy.

            Parameters
            ----------
            tiles : list of morecantile.Tile
                The list of tiles to create parent tiles for.
        """

        tile_manager = self.tiles
        config_manager = self.config

        tileset_objs = []

        # Make the next level of parent tiles
        for parent_tile in tiles:
            # Get the path to the parent tile
            parent_path = tile_manager.path_from_tile(parent_tile, '3dtiles')
            # Get just the base dir without the filename
            parent_dir = os.path.dirname(parent_path)
            # Get the filename of the parent tile, without the extension
            parent_filename = os.path.basename(parent_path)
            parent_filename = os.path.splitext(parent_filename)[0]
            # Get the children paths for this parent tile
            child_paths = tile_manager.get_child_paths(parent_tile, '3dtiles')
            # Remove paths that do not exist
            child_paths = tile_manager.remove_nonexistent_paths(child_paths)
            # Get the parent bounding volume
            parent_bv = self.bounding_region_for_tile(
                parent_tile, limit_to=bv_limit)
            # If the bounding region is outside t
            # Get the version
            version = config_manager.get('version')
            # Get the geometric error
            geometric_error = config_manager.get('geometricError')
            # Create the parent tile
            tileset_obj = TreeGenerator.parent_tile_from_children_json(
                child_paths,
                dir=parent_dir,
                filename=parent_filename,
                geometricError=geometric_error,
                tilesetVersion=version,
                boundingVolume=parent_bv
            )
            tileset_objs.append(tileset_obj)

        return tileset_objs

    def bounding_region_for_tile(self, tile, limit_to=None):
        """
        For a morecantile.Tile object, return a BoundingVolumeRegion object
        that represents the bounding region of the tile.

        Parameters
        ----------
        tile : morecantile.Tile
            The tile object.
        limit_to : list of float
            Optional list of west, south, east, north coordinates to limit
            the bounding region to.

        Returns
        -------
        bv : BoundingVolumeRegion
            The bounding region object.
        """
        tms = self.tiles.tms
        bounds = tms.bounds(tile)
        bounds = gpd.GeoSeries(
            box(bounds.left, bounds.bottom, bounds.right, bounds.top),
            crs=tms.crs)
        if limit_to is not None:
            bounds_limitor = gpd.GeoSeries(
                box(limit_to[0], limit_to[1], limit_to[2], limit_to[3]),
                crs=tms.crs)
            bounds = bounds.intersection(bounds_limitor)
        bounds = bounds.to_crs(BoundingVolumeRegion.CESIUM_EPSG)
        bounds = bounds.total_bounds

        region_bv = {
            'west': bounds[0], 'south': bounds[1],
            'east': bounds[2], 'north': bounds[3],
        }
        return region_bv

### Configuration files and creating stager, rasterizer, and tilers

In [5]:
# staging configuration
stager = pdgstaging.TileStager(workflow_config)
tile_manager = stager.tiles
config_manager = stager.config

# zoom levels configuration
min_z = config_manager.get_min_z()
max_z = config_manager.get_max_z()
parent_zs = range(max_z - 1, min_z - 1, -1)

# 3D tiler configuration
tiles3dmaker = StagedTo3DConverter(workflow_config)

# raster tilerconfiguration 
#rasterizer = pdgraster.RasterTiler(workflow_config)

### Parsl setup

In [6]:
# bash command to activate virtual environment
activate_env = 'source /home/jcohen/.bashrc; conda activate pdgWorkflow'

htex_config_local = Config(
  executors = [
      HighThroughputExecutor(
        label = "htex_Local",
        cores_per_worker = 2, 
        max_workers = 2, # why would this be so low? because just testing with small amount of data ?
          # worker_logdir_root = '/' only necessary if the file system is remote, which is not the case for this lake change sample
          # address not necessary because we are not using kubernetes
        worker_debug = False, # don't need this because we have logging setup
          # provider is local for this run thru, kubernetes would use KubernetesProvider()
        provider = LocalProvider(
          channel = LocalChannel(),
          worker_init = activate_env,
          init_blocks = 1, # default I think
          max_blocks = 10 # changed from deafult of 1
        ),
      )
    ],
  )

parsl.clear() # first clear the current configuration since we will likely run this script multiple times
parsl.load(htex_config_local) # load the config we just outlined

### Batching setup

In [7]:
batch_size_staging=1 # change this depending on data sample size!!!!!
batch_size_rasterization=30
batch_size_3dtiles=20
batch_size_parent_3dtiles=500
batch_size_geotiffs=200
batch_size_web_tiles=200

In [8]:
def make_batch(items, batch_size):
    """
    Create batches of a given size from a list of items.
    """
    return [items[i:i + batch_size] for i in range(0, len(items), batch_size)]

### Staging

In [ ]:
#input_batches = make_batch(data_paths, batch_size_staging)
#input_batches # 3 batches, 1 file each

In [ ]:
# # Decorators seem to be ignored as the first line of a cell, so print something first
# print("Stage in parallel")

# @python_app
# def stage(paths, config, logging_dict = logging_dict): 
#     """
#     Stage files (step 1)
#     """
#     import pdgstaging
#     if logging_dict:
#         import logging.config
#         logging.config.dictConfig(logging_dict)
#     stager = pdgstaging.TileStager(config)
#     for path in paths:
#         stager.stage(path)
#     return True

In [ ]:
# app_futures = []
# for batch in input_batches:
#     app_future = stage(batch, workflow_config, logging_dict)
#     app_futures.append(app_future)

# [a.result() for a in app_futures]

In [ ]:
# htex_config_local.executors[0].shutdown()
# parsl.clear()

### Rasterization

In [9]:
# Get paths to all the newly staged tiles
staged_paths = stager.tiles.get_filenames_from_dir('staged')

In [ ]:
# len(staged_paths)

In [10]:
# batch staged files
staged_batches = make_batch(staged_paths, batch_size_rasterization)
len(staged_batches) # 637 batches

637

In [ ]:
# see what is within 1 batch
# staged_batches[0]

In [11]:
print('rasterize in parallel')

@python_app
def rasterize(staged_paths, config, logging_dict = logging_dict):
    """
    Rasterize a batch of vector files (step 2)
    """
    import pdgraster
    if logging_dict:
        import logging.config
        logging.config.dictConfig(logging_dict)
    rasterizer = pdgraster.RasterTiler(config)
    return rasterizer.rasterize_vectors(staged_paths, make_parents = True)

rasterize in parallel


In [12]:
app_futures = []
for batch in staged_batches:
    app_future = rasterize(batch, workflow_config, logging_dict)
    app_futures.append(app_future)

# Don't continue to step 3 until all tiles have been rasterized
[a.result() for a in app_futures]

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,

In [13]:
htex_config_local.executors[0].shutdown()
parsl.clear()

In [ ]:
# staged file paths = 19088, geotiff z-level 11 = 19085, 3 missing
# get raster file paths from z-level 11 to compare to staged file paths

# start with all z-levels because the input to this function needs to be base dir
geotiff_paths = tile_manager.get_filenames_from_dir('geotiff')

# remove strings that do not have z-level 11
geotiff_paths_11 = []
for path in geotiff_paths:
    if '1' == path[21:22]:
        geotiff_paths_11.append(path)

#geotiff_paths_11

# remove leading dir 'geotiff'
geotiff_paths_11_trimmed = []
for path in geotiff_paths_11:
    trimmed_1 = path.replace('geotiff', '')
    trimmed_2 = trimmed_1.replace('.tif', '')
    geotiff_paths_11_trimmed.append(trimmed_2)

geotiff_paths_11_trimmed[0:10]


In [ ]:
staged_paths = stager.tiles.get_filenames_from_dir('staged')

# remove 'staged' and 'gpkg'
staged_paths_trimmed = []
for path in staged_paths:
    trimmed_1 = path.replace('staged', '')
    trimmed_2 = trimmed_1.replace('.gpkg', '')
    staged_paths_trimmed.append(trimmed_2)

staged_paths_trimmed[0:10]



In [ ]:
# subtract lists of file paths
missing_files = []
for path in staged_paths_trimmed:
  if path not in geotiff_paths_11_trimmed:
    missing_files.append(path)

missing_files

### Rasterize problematic files on their own with modified viz-staging (edited locally)

In [ ]:
base_dir = 'staged'
ext = '.gpkg'
problem_file_1 = base_dir + '/WGS1984Quad/11/557/256' + ext
problem_file_2 = base_dir + '/WGS1984Quad/11/463/320' + ext
problem_file_3 = base_dir + '/WGS1984Quad/11/463/253' + ext

# make into list
prob_files = [problem_file_1, problem_file_2, problem_file_3]

# make batches (1 file per batch)
prob_batches = make_batch(prob_files, 1)
prob_batches[0]

In [ ]:
app_futures = []
for batch in prob_batches:
    app_future = rasterize(batch, workflow_config, logging_dict)
    app_futures.append(app_future)

# Don't continue to step 3 until all tiles have been rasterized
[a.result() for a in app_futures]

#### Check the following:

1. number of rasters in z-level 11 compared to number of staged files (which are all z-level 11)
2. number of rasters in all z-levels matches number of rasters in all z-levels from other run-through not in parallel (lake_change_sample dir)
3. any weird formatting in rasters_summary.csv? fix if so
4. any errors reported in rasterization_events.csv?
4. any errors reported in log.log?

### Create Web Tiles from geoTIFF's

In [ ]:
# # Update ranges from raster_summary.csv
# rasterizer.update_ranges()

In [ ]:
# Process web tiles in batches
# geotiff_batches = make_batch(geotiff_paths, batch_size_web_tiles)